<a href="https://colab.research.google.com/github/Seungminin/DeepFake_detection_SING/blob/Yujin/sing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import glob
import zipfile
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os
import shutil
import pandas as pd
import random
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from torchvision import transforms
from torch import nn
from torchvision import models
import sys
import seaborn as sn
from sklearn.metrics import confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive

!unzip -qq "/content/drive/MyDrive/sing.zip"

/content/drive/MyDrive


In [5]:
test_fake_images = list(glob.glob('/content/drive/MyDrive/real_vs_fake/real-vs-fake/test/fake/*.jpg'))
test_real_images = list(glob.glob('/content/drive/MyDrive/real_vs_fake/real-vs-fake/test/real/*.jpg'))

print(len(test_fake_images))
print(len(test_real_images))

10000
10000


In [6]:
train_fake_images = list(glob.glob('/content/drive/MyDrive/real_vs_fake/real-vs-fake/train/fake/*.jpg'))
train_real_images = list(glob.glob('/content/drive/MyDrive/real_vs_fake/real-vs-fake/train/real/*.jpg'))

print(len(train_fake_images))
print(len(train_real_images))

50000
50000


In [8]:
def resize_images_in_folder(image_folder, target_size=(224, 224)):
    for filename in os.listdir(image_folder):
        file_path = os.path.join(image_folder, filename)
        try:
            img = Image.open(file_path)
            img_resized = img.resize(target_size, Image.ANTIALIAS)
            img_resized.save(file_path)  # 기존 이미지를 덮어쓰기
        except (IOError, SyntaxError) as e:
            print(f"Error processing image: {file_path}")
            continue

# 테스트 이미지 크기 조정
resized_real_images = resize_images_in_folder('/content/drive/MyDrive/real_vs_fake/real-vs-fake/test/real/', target_size=(224, 224))
resized_fake_images = resize_images_in_folder('/content/drive/MyDrive/real_vs_fake/real-vs-fake/test/fake/',target_size=(224, 224))

<ipython-input-8-03f3015e3f06>:6: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize(target_size, Image.ANTIALIAS)


In [11]:
test_fake_images = list(glob.glob('/content/drive/MyDrive/real_vs_fake/real-vs-fake/test/fake/*.jpg'))
test_real_images = list(glob.glob('/content/drive/MyDrive/real_vs_fake/real-vs-fake/test/real/*.jpg'))

print(len(test_fake_images))
print(len(test_real_images))

10000
10000


In [ ]:
# # 원본 파일이 위치한 폴더
# source_folder = '/content/drive/MyDrive/sing'

# # 백업할 위치의 폴더
# backup_folder = '/content/drive/MyDrive/backup'

# # 백업할 파일의 확장자를 지정
# file_extension = '*.jpg'

# # 원본 폴더에서 해당 확장자를 가진 모든 파일의 경로를 찾음
# files_to_backup = glob.glob(os.path.join(source_folder, file_extension))

# # 각 파일을 백업 폴더로 복사
# for file_path in files_to_backup:
#     # 원본 파일의 이름을 가져옴
#     file_name = os.path.basename(file_path)
#     # 백업 파일의 전체 경로를 생성
#     backup_path = os.path.join(backup_folder, file_name)
#     # 파일을 백업 폴더로 복사
#     shutil.copy(file_path, backup_path)
#     print(f'{file_path} -> {backup_path}')

# print("백업 완료.")

In [ ]:
# image_paths = '/content/drive/MyDrive/sing/~~/' # test_fake_images, test_real_images, train_fake_images, train_real_images

# def validate_image(image_paths):
#     try:
#         img = Image.open(image_paths) # 이미지 열기 시도
#         img.verify() # 이미지 검증 시도
#         return True # 검증 성공 시 True 반환
#     except (IOError, SyntaxError) as e:
#         print('손상된 이미지:', image_paths)
#         return False # 검증 실패 시 False 반환

# # fake 이미지 파일 경로들을 수집
# image_files = glob.glob('/content/drive/MyDrive/sing/~~/*.jpg')
# print("총 이미지 파일 수:", len(image_files))

# # 손상된 이미지 파일들을 삭제
# for image_paths in image_files:
#     if not validate_image(image_paths):
#         os.remove(image_paths) # 손상된 이미지 삭제
#         print(f'{image_paths} 삭제됨')

In [ ]:
# # CSV 파일 로드
# df = pd.read_csv('/content/drive/MyDrive/test.csv')

# # 이미지와 레이블 리스트 준비
# images = []
# labels = []

# # 이미지 로드 및 처리
# for index, row in df.iterrows():
#     img_path = os.path.join('/content/drive/MyDrive/modify_test', row['path'])  # 이미지 경로 조합
#     try:
#         img = Image.open(img_path)
#         img_array = np.array(img)  # 이미지를 numpy 배열로 변환
#         images.append(img_array)  # 변환된 이미지 배열 추가
#         labels.append(row['label'])  # 레이블 추가
#     except IOError:
#         print(f"Error loading image: {img_path}")  # 이미지 로드 실패 시 오류 메시지 출력

# print(f"Total number of images loaded: {len(images)}")

In [ ]:
# # 이미지를 읽고 시각화하는 함수
# def im_plot(image):
#     b,g,r = cv2.split(image)
#     image = cv2.merge((r,g,b))
#     image = image*[0.22803, 0.22145, 0.216989] +  [0.43216, 0.394666, 0.37645]
#     image = image*255.0
#     plt.imshow(image.astype(int))
#     plt.show()

# # 이미지 파일을 하나씩 읽어서 처리
# for image_files in image_files:
#     image = cv2.imread(image_files)
#     im_plot(image)

In [24]:
class ImageDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.data_frame.iloc[idx, 5]  # 이미지 파일 경로
        image = Image.open(img_name)
        label = self.data_frame.iloc[idx, 3]  # 라벨 정보

        if self.transform:
            image = self.transform(image)

        return image, label

# 이미지 전처리를 위한 transforms 정의
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 데이터셋 인스턴스 생성
train_dataset = ImageDataset(csv_file='/content/drive/MyDrive/train.csv', transform=transform)
valid_dataset = ImageDataset(csv_file='/content/drive/MyDrive/valid.csv', transform=transform)
test_dataset = ImageDataset(csv_file='/content/drive/MyDrive/test.csv', transform=transform)

# DataLoader 인스턴스 생성
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [25]:
def number_of_real_and_fake_images(data_list):
    # CSV 파일로부터 데이터 불러오기, 첫 번째 행에 열 이름이 포함되어 있으므로 header=0을 사용
    labels = pd.read_csv('/content/drive/MyDrive/test.csv', header=0)
    fake = 0
    real = 0
    for image_path in data_list:
        image_name = image_path.split('/')[-1]
        # 'path' 열에서 파일 이름을 포함하는 행 찾기
        label_row = labels[labels['path'].str.contains(image_name)]
        if not label_row.empty:
            label = label_row['label'].values[0]
            if label == 0:  # fake
                fake += 1
            elif label == 1:  # real
                real += 1
    return real, fake

In [29]:
import pandas as pd
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# CSV 파일 로드
train_df = pd.read_csv('/content/drive/MyDrive/train.csv', skiprows=1, header=None, names=['index', 'original_path', 'id', 'label', 'label_str', 'path'])
valid_df = pd.read_csv('/content/drive/MyDrive/valid.csv', skiprows=1, header=None, names=['index', 'original_path', 'id', 'label', 'label_str', 'path'])

# 이미지 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.dataframe.iloc[idx, 5])
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx, 3]

        if self.transform:
            image = self.transform(image)

        return image, label

# 이미지 전처리 정의
im_size = 112
train_transforms = transforms.Compose([
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 데이터셋 생성
train_data = ImageDataset(train_df, transform=train_transforms)
val_data = ImageDataset(valid_df, transform=test_transforms)

# DataLoader 생성
train_loader = DataLoader(train_data, batch_size=4, shuffle=True, num_workers=4)
valid_loader = DataLoader(val_data, batch_size=4, shuffle=True, num_workers=4)

# 데이터 확인 (선택사항)
# 이미지와 라벨을 하나 출력해보기
import matplotlib.pyplot as plt

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

# 첫 번째 배치의 이미지와 라벨을 가져와서 출력해봅니다.
images, labels = next(iter(train_loader))
out = torchvision.utils.make_grid(images)
imshow(out, title=[str(x.item()) for x in labels])


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-29-ebbecc4c7f76>", line 23, in __getitem__
    image = Image.open(img_name)
  File "/usr/local/lib/python3.10/dist-packages/PIL/Image.py", line 3227, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'train/fake/CUMJCHGEO8.jpg'


In [18]:
class Model(nn.Module):
    def __init__(self, num_classes):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.model(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dp(self.linear1(x))
        return x

In [19]:
# 모델에 적합한 입력 형태로 텐서 생성
correct_shape_tensor = torch.randn(1, 3, 112, 112).cuda()  # 무작위 데이터로 채워진 텐서 생성

# 모델에 입력 데이터 전달 및 예측 수행
a, b = model(correct_shape_tensor)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [20]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def calculate_accuracy(outputs, targets):
    batch_size = targets.size(0)
    _, pred = outputs.topk(1, 1, True)
    pred = pred.t()
    correct = pred.eq(targets.view(1, -1))
    n_correct_elems = correct.float().sum().item()

    return 100 * n_correct_elems / batch_size

def train_epoch(epoch, num_epochs, data_loader, model, criterion, optimizer):
    model.train()
    losses = AverageMeter()
    accuracies = AverageMeter()

    for i, (inputs, targets) in enumerate(data_loader):
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda().type(torch.cuda.LongTensor)

        outputs = model(inputs)  # 가정: 모델은 하나의 출력만 반환합니다.
        loss = criterion(outputs, targets)
        acc = calculate_accuracy(outputs, targets)

        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d / %d] [Loss: %f, Acc: %.2f%%]"
                % (
                    epoch,
                    num_epochs,
                    i,
                    len(data_loader),
                    losses.avg,
                    accuracies.avg))

    torch.save(model.state_dict(), '/content/checkpoint.pt')

    return losses.avg, accuracies.avg

def test(epoch, model, data_loader, criterion):
    print('Testing')
    model.eval()
    losses = AverageMeter()
    accuracies = AverageMeter()
    pred = []
    true = []

    with torch.no_grad():
        for i, (inputs, targets) in enumerate(data_loader):
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                targets = targets.cuda().type(torch.cuda.LongTensor)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            acc = calculate_accuracy(outputs, targets)

            _, p = torch.max(outputs, 1)
            true += targets.detach().cpu().numpy().tolist()
            pred += p.detach().cpu().numpy().tolist()

            losses.update(loss.item(), inputs.size(0))
            accuracies.update(acc, inputs.size(0))

            sys.stdout.write(
                    "\r[Batch %d / %d]  [Loss: %f, Acc: %.2f%%]"
                    % (
                        i,
                        len(data_loader),
                        losses.avg,
                        accuracies.avg
                        ))

        print('\nAccuracy {}'.format(accuracies.avg))

    return true, pred, losses.avg, accuracies.avg

In [ ]:
#Output confusion matrix
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[0][0])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[1][1])
    print('\n')
    df_cm = pd.DataFrame(cm, range(2), range(2))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
    plt.ylabel('Actual label', size = 20)
    plt.xlabel('Predicted label', size = 20)
    plt.xticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.yticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.ylim([2, 0])
    plt.show()
    calculated_acc = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+ cm[1][1])
    print("Calculated Accuracy",calculated_acc*100)

In [ ]:
def plot_loss(train_loss_avg,test_loss_avg,num_epochs):
  loss_train = train_loss_avg
  loss_val = test_loss_avg
  print(num_epochs)
  epochs = range(1,num_epochs+1)
  plt.plot(epochs, loss_train, 'g', label='Training loss')
  plt.plot(epochs, loss_val, 'b', label='validation loss')
  plt.title('Training and Validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()
def plot_accuracy(train_accuracy,test_accuracy,num_epochs):
  loss_train = train_accuracy
  loss_val = test_accuracy
  epochs = range(1,num_epochs+1)
  plt.plot(epochs, loss_train, 'g', label='Training accuracy')
  plt.plot(epochs, loss_val, 'b', label='validation accuracy')
  plt.title('Training and Validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

In [ ]:
#learning rate
lr = 1e-5#0.001
#number of epochs
num_epochs = 20

optimizer = torch.optim.Adam(model.parameters(), lr= lr,weight_decay = 1e-5)

#class_weights = torch.from_numpy(np.asarray([1,15])).type(torch.FloatTensor).cuda()
#criterion = nn.CrossEntropyLoss(weight = class_weights).cuda()
criterion = nn.CrossEntropyLoss().cuda()
train_loss_avg =[]
train_accuracy = []
test_loss_avg = []
test_accuracy = []
for epoch in range(1,num_epochs+1):
    l, acc = train_epoch(epoch,num_epochs,train_loader,model,criterion,optimizer)
    train_loss_avg.append(l)
    train_accuracy.append(acc)
    true,pred,tl,t_acc = test(epoch,model,valid_loader,criterion)
    test_loss_avg.append(tl)
    test_accuracy.append(t_acc)
plot_loss(train_loss_avg,test_loss_avg,len(train_loss_avg))
plot_accuracy(train_accuracy,test_accuracy,len(train_accuracy))
print(confusion_matrix(true,pred))
print_confusion_matrix(true,pred)